In [2]:
import nltk
from nltk.corpus import twitter_samples 
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install tensor
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [3]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [7]:
#process_tweet(): cleans the text, tokenizes it into separate words, removes stopwords, and converts words to stems.
def process_tweet(tweet):
   
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
 
    tweet = re.sub(r'#', '', tweet)
   


    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
            
           
            # 1 remove stopwords
            if(word not in stopwords_english and word not in string.punctuation):
              steam_word=stemmer.stem(word)
              tweets_clean.append(steam_word)
            # 3 stemming word
            # 4 Add it to tweets_clean
    return tweets_clean

In [8]:


def build_freqs(tweets, ys):

    yslist = np.squeeze(ys).tolist()
    freqs = {}

    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair not in freqs:
              freqs[pair]=0
            freqs[pair]+=1


    return freqs

In [15]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos=all_positive_tweets[4000:]
train_pos=all_positive_tweets[:4000]
test_neg=all_negative_tweets[4000:]
train_neg=all_negative_tweets[:4000]
train_x=train_pos+train_neg
test_x=test_pos+test_neg

In [16]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [33]:
data=all_positive_tweets+all_negative_tweets
la=np.append(np.ones((len(all_positive_tweets), 1)), np.zeros((len(all_negative_tweets), 1)), axis=0)
train_x,test_x,train_y,test_y=train_test_split(data,la,test_size=0.30,random_state=68)

In [34]:
# create frequency dictionary

freqs = build_freqs(train_x,train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 10312


In [35]:
#############################################################
### ------Process tweet-----------
# Example
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 @WillHillBet thank you for taking the time to reply, albeit with a disappointing answer - the most famous athlete racing and no odds :(

This is an example of the processed version of the tweet: 
 ['thank', 'take', 'time', 'repli', 'albeit', 'disappoint', 'answer', 'famou', 'athlet', 'race', 'odd', ':(']


In [36]:
#Extracting the features
def extract_features(tweet, freqs):
 
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 2)) 
    
    #bias term is set to 1
    # x[0,0] = 1 
        
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        if((word,1) in freqs):
          x[0,0]+=freqs[word,1]
        
        # increment the word count for the negative label 0
        if((word,0) in freqs):
          x[0,1]+=freqs[word,0]
    
    assert(x.shape == (1, 2))
    return x[0]

In [37]:
# Check the function

# test 1
# test on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[ 573. 3425.]


In [38]:
# test 2:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('happy', freqs)
print(tmp2)

[147.  18.]


In [39]:
def predict_tweet(tweet):

    with tf.Session() as sess:
      saver.restore(sess,save_path='TSession')
      data_i=[]
      for t in tweet:
        data_i.append(extract_features(t,freqs))
      data_i=np.asarray(data_i)
      return sess.run(tf.nn.sigmoid(tf.add(tf.matmul(a=data_i,b=W,transpose_b=True),b)))
    print("Fail")
    return 
 

In [40]:
#Define bias and Weight

b=tf.Variable(np.random.randn(1),name="Bias")
W=tf.Variable(np.random.randn(1,2),name="Bias")

In [41]:
#Extract Feature for all tweets

data=[]
for t in train_x:
  data.append(extract_features(t,freqs))
data=np.asarray(data)

In [42]:
#Define Sigmoid Function and Cost Function
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(np.asarray(data), W,transpose_b=True), b)) 
print(Y_hat)
ta=np.asarray(train_y)
# Sigmoid Cross Entropy Cost Function 
cost = tf.nn.sigmoid_cross_entropy_with_logits( 
                    logits = Y_hat, labels = ta) 
print(cost)

Tensor("Sigmoid_2:0", shape=(7000, 1), dtype=float64)
Tensor("logistic_loss_1:0", shape=(7000, 1), dtype=float64)


In [43]:
#Define Gradient Descent Optimizer
  
# Gradient Descent Optimizer 
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-4,name="GradientDescent").minimize(cost) 
  
# Global Variables Initializer 
init = tf.global_variables_initializer() 

In [44]:
#Run Epoch for 1000 times

saver = tf.train.Saver()
with tf.Session() as sess:
  
  sess.run(init)
  print("Bias",sess.run(b))
  print("Weight",sess.run(W))
  for epoch in range(400):
    sess.run(optimizer)
    preds=sess.run(Y_hat)
    acc=((preds==ta).sum())/len(train_y)
    accu=[]
    repoch=False
    if repoch:
      accu.append(acc)
    if epoch % 1000 == 0:
      print("Accuracy",acc)
    saved_path = saver.save(sess, 'TSession')

Bias [0.89781018]
Weight [[0.70234961 0.78590555]]
Accuracy 0.49857142857142855


In [45]:
#Predict All Tweet

preds=predict_tweet(test_x)
print(preds,len(test_y))

INFO:tensorflow:Restoring parameters from TSession
[[1.        ]
 [1.        ]
 [1.        ]
 ...
 [1.        ]
 [1.        ]
 [0.99999834]] 3000


In [46]:
#Calculate Accuracy

def calculate_accuracy(x,y):
  if len(x)!=len(y):
    print("dimensions are different")
    return
  return ((x==y).sum())/len(y)

In [47]:
print(calculate_accuracy(preds,test_y))

0.492
